In [1]:
import requests
import requesocks
import json
#import grab
import logging
import codecs
from lxml.etree import ElementTree, fromstring
from bs4 import BeautifulSoup
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from IPython.display import HTML
import sqlite3 as sq
import pandas as pd
from stem import Signal
from stem.control import Controller
import numpy as np
import re
from torrequest import TorRequest
import time
import datetime
from datetime import date
import zipfile

In [2]:
def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {'http':  'socks5://127.0.0.1:9050',
                       'https': 'socks5://127.0.0.1:9050'}
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:56.0) Gecko/20100101 Firefox/56.0'
    })
    
    return session

# signal TOR for a new connection 
def renew_connection():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="3246")
        controller.signal(Signal.NEWNYM)
        
def get_my_ip(session):
    text = session.get("https://httpbin.org/ip").text
    ipjson = json.loads(text)
    return ipjson['origin']

In [3]:
def create_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
#     session.proxies = {'http':  'socks5://127.0.0.1:9050',
#                        'https': 'socks5://127.0.0.1:9050'}
#     session.headers.update({
#         'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:56.0) Gecko/20100101 Firefox/56.0'
#     })
    return session

In [4]:
MY_KEY = "0e7262816e22e8ec"

emails = ['iuqrr@hi2.in', 'pitbox531@gmail.com', 'pitbox96_t1@outlook.com', 'pitbox96_t2@mail.ru','pitbox96_t3@mail.ru']
keys = ['11fa55c46ce6fb9c', '0e7262816e22e8ec', '5c9f96bf5020569e', 'c3a1b0883114ce1c', '38e6ac0087e81d0f']

# date_str = "20060405"
#location_str = "NY/New_York" 
location_str = "DC/Washington"
loc_short = "DC_Washington"

# url = "http://api.wunderground.com/api/{0}/history_{1}/q/{2}.json".format(MY_KEY,date_str ,location_str)

# session = create_session()
# json_text = session.get(url).text

# print json_text

def get_json_request(date, location, api_key, session):
    date_str = date.strftime('%Y%m%d')
    url = "http://api.wunderground.com/api/{0}/history_{1}/q/{2}.json".format(api_key, date_str ,location)
    json_text = session.get(url).text
    
    #json_res = json.loads(json_text)
    return json_text
    

In [18]:
from tqdm import tnrange, tqdm_notebook
from time import sleep

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + datetime.timedelta(n)
        
        
def download_weather():
    st_date = datetime.date(2012, 1, 1)
    
    st_date += datetime.timedelta(1800)
    
    #cur_date = st_date  
    end_date = datetime.date(2017, 11, 20)
    
    dates = [x for x in daterange(st_date, end_date)]

    df = pd.DataFrame(columns = ['loc','date', 'json'])

    res_data = []

    idx = 1
    
    key_index = 2
    
    for cur_date in tqdm_notebook(dates):
        session = create_session()
        cur_json_text = get_json_request(cur_date, location_str, keys[key_index], session)
        
        row = {'loc': location_str, 'date':str(cur_date), 'json': cur_json_text}
        
        res_data.append(row)
        
        #cur_date += datetime.timedelta(days=1)

        if idx % 100 == 0:
            print str(idx), " saved! : ", str(cur_date)
            df1 = pd.DataFrame(res_data)
            df1.to_csv("result_"+str(idx)+"_" + loc_short +".gz", compression="gzip")
            #key_index = (key_index + 1) % len(keys)  

        idx += 1


    df = pd.DataFrame(res_data)
    df.to_csv("result_"+ loc_short + ".gz", compression="gzip")

download_weather()

100  saved! :  2017-03-14
200  saved! :  2017-06-22
300  saved! :  2017-09-30



In [16]:


# st_date = datetime.date(2012, 1, 1)
# cur_date = st_date  
# end_date = datetime.date(2012, 1, 20)

# dates = daterange(st_date, end_date)
# list1 =  [x for x in dates]
# list1

st_date = datetime.date(2012, 1, 1)
st_date += datetime.timedelta(1800)
st_date

datetime.date(2016, 12, 5)

In [49]:
df_new = pd.read_csv("sample_result_NY.gz", compression="gzip")
df_new.head()

,Unnamed: 0,date,json,loc
0,0,2006-04-05,"\n{\n ""response"": {\n ""version"":""0.1"",\n ""t...",NY
1,1,2006-04-06,"\n{\n ""response"": {\n ""version"":""0.1"",\n ""t...",NY
2,2,2006-04-07,"\n{\n ""response"": {\n ""version"":""0.1"",\n ""t...",NY
3,3,2006-04-08,"\n{\n ""response"": {\n ""version"":""0.1"",\n ""t...",NY
4,4,2006-04-09,"\n{\n ""response"": {\n ""version"":""0.1"",\n ""t...",NY
